# Read the database

In [58]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


./data\database.sqlite
./data\dataNotes.md


We will now move on to the first round of data processing, where values which the full time result does not exist are removed.

In [59]:
import sqlite3
import sklearn, numpy
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

con = sqlite3.connect('./data/database.sqlite')

df = pd.read_sql_query("select * from football_data;", con)
#df = df.sample(frac=0.01)

print("Initial dataframe shape:", df.shape)

index_names = df[(df['FTR'] != 'H') & (df['FTR'] != 'A') & (df['FTR'] != 'D')].index

#df = df.sort_values('Datetime')

df.drop(index_names, inplace=True)

column_names = list(df.columns.values)
#df.dropna(inplace=True, thresh=int(0.8*df.shape[0]) , axis=1)

Initial dataframe shape: (179807, 172)


# Data Preprocess

## Definition of data

### Match Information
- Div = League Division
- Date = Match Date (dd/mm/yy)
- Time = Time of match kick off
- HomeTeam = Home Team
- AwayTeam = Away Team
- FTHG and HG = Full Time Home Team Goals
- FTAG and AG = Full Time Away Team Goals
- FTR and Res = Full Time Result (H=Home Win, D=Draw, A=Away Win)
- HTHG = Half Time Home Team Goals
- HTAG = Half Time Away Team Goals
- HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)

### Match Statistics (where available)
- Attendance = Crowd Attendance
- Referee = Match Referee
- HS = Home Team Shots
- AS = Away Team Shots
- HST = Home Team Shots on Target
- AST = Away Team Shots on Target
- HHW = Home Team Hit Woodwork
- AHW = Away Team Hit Woodwork
- HC = Home Team Corners
- AC = Away Team Corners
- HF = Home Team Fouls Committed
- AF = Away Team Fouls Committed
- HFKC = Home Team Free Kicks Conceded
- AFKC = Away Team Free Kicks Conceded
- HO = Home Team Offsides
- AO = Away Team Offsides
- HY = Home Team Yellow Cards
- AY = Away Team Yellow Cards
- HR = Home Team Red Cards
- AR = Away Team Red Cards
- HBP = Home Team Bookings Points (10 = yellow, 25 = red)
- ABP = Away Team Bookings Points (10 = yellow, 25 = red)

## Data Wrangling

In [60]:
# These attributes contain significant number of null values

noneSet = []
noneSet = [
    'ABP', 'AFKC', 'AHW', 'AO', 'AT', 'Attendance', 'BSA', 'BSD', 'BSH',
    'Bb1X2', 'BbAH', 'BbAHh', 'BbAv<2.5', 'BbAv>2.5', 'BbAvA', 'BbAvAHA',
    'BbAvAHH', 'BbAvD', 'BbAvH', 'BbMx<2.5', 'BbMx>2.5', 'B365AH', 'BbMxA',
    'BbMxAHA', 'BbMxAHH', 'BbMxD', 'BbMxH', 'BbOU', 'GB<2.5', 'GB>2.5', 'GBA',
    'GBAH', 'GBAHA', 'GBAHH', 'GBD', 'GBH', 'HBP', 'HFKC', 'HHW', 'HO', 'HT',
    'LBAH', 'LBAHA', 'LBAHH', 'SBA', 'SBD', 'SBH', 'SJA', 'SJD', 'SJH', 'SOA',
    'SOD', 'SOH', 'SYA', 'SYD', 'SYH', 'PA', 'PD', 'PH', 'LBH', 'LBD', 'LBA'
]

allow_halftime = False

# Result, can not be used for prediction
results = ['FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR']

# The value of these columns are not known before the match as well
match_statistics = [
    'Attendance', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HHW', 'AHW', 'HC',
    'AC', 'HF', 'AF', 'HFKC', 'AFKC', 'HO', 'AO', 'HY', 'AY', 'HR', 'AR',
    'HBP', 'ABP'
]

X = df.drop(set(results + match_statistics +
                ['Datetime', 'Season', 'HT', 'AT']).intersection(column_names),
            axis=1)
print(X.shape)



(179803, 140)


## Converting the datetime

In [61]:
import datetime, time


# https://stackoverflow.com/questions/57330482/convert-data-frame-datatime-string-to-float-in-python-pandas
def convertDate(dateString):
    dateTime1 = datetime.datetime.strptime(dateString, '%Y-%m-%d %H:%M:%S')
    #ignores time of the day
    return int(time.mktime(dateTime1.timetuple())) // 86400


def convertTime(timeString):
    hour, minute = map(int, timeString.split(':'))
    return hour * 60 + minute


def reciprocal(x):
    if x == 0:
        return 0
    return 1 / float(x)


df['Date'] = df['Date'].apply(convertDate)
df['Time'] = df['Time'].apply(convertTime)

In [55]:
df[['Date','Time','Datetime']]

,Date,Time,Datetime
0,18494,30,2020-08-21 00:30:00
1,18494,30,2020-08-21 00:30:00
2,18494,0,2020-08-21 00:00:00
3,18494,0,2020-08-21 00:00:00
4,18494,0,2020-08-21 00:00:00
...,...,...,...
179802,11166,0,2000-07-29 00:00:00
179803,11166,0,2000-07-29 00:00:00
179804,11165,0,2000-07-28 00:00:00
179805,11165,0,2000-07-28 00:00:00


## Fill the Nan in odds and calculate reciprocal

In [63]:
# odds
betting_odds = [
    'B365H', 'B365D', 'B365A', 'BSH', 'BSD', 'BSA', 'BWH', 'BWD', 'BWA', 'GBH',
    'GBD', 'GBA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PH', 'PD', 'PA',
    'SOH', 'SOD', 'SOA', 'SBH', 'SBD', 'SBA', 'SJH', 'SJD', 'SJA', 'SYH',
    'SYD', 'SYA', 'VCH', 'VCD', 'VCA', 'WHH', 'WHD', 'WHA', 'Bb1X2', 'BbMxH',
    'BbAvH', 'BbMxD', 'BbAvD', 'BbMxA', 'BbAvA', 'MaxH', 'MaxD', 'MaxA',
    'AvgH', 'AvgD', 'AvgA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD',
    'BWCA', 'IWCH', 'IWCD', 'IWCA', 'VCCH', 'VCCD', 'VCCA', 'WHCH', 'WHCD',
    'WHCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD', 'AvgCA', 'PSH', 'PSD',
    'PSA', 'PSCA', 'PSCD'
]

# goals related to goals
num_goals_odds = [
    'BbMx>2.5', 'BbAv>2.5', 'BbMx<2.5', 'BbAv<2.5', 'GB>2.5', 'GB<2.5',
    'B365>2.5', 'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5',
    'Avg<2.5', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5',
    'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5'
]

# odds in asia
asian_odds = [
    'AHh', 'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA', 'GBAHH', 'GBAHA',
    'GBAH', 'LBAHH', 'LBAHA', 'LBAH', 'B365AHH', 'B365AHA', 'B365AH', 'PAHH',
    'PAHA', 'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA', 'B365CAHH', 'B365CAHA',
    'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA'
]

# Fill the NA with the mean
for i in betting_odds:
    try:
        df[i].fillna((df[i].mean()), inplace=True)
    except:
        print(i)

for i in num_goals_odds + asian_odds:
    df[i].fillna((df[i].mean()), inplace=True)

column_names = list(df.columns.values)

#Replace empty values with -1
for i in column_names:
    df[i].fillna(-1, inplace=True)

# convert the fraction to float(1/x->x)
for col in betting_odds + num_goals_odds + asian_odds:
    try:
        df[col] = df[col].apply(reciprocal)
    except KeyError:
        continue
        #print(col, "KeyError")
    except:
        print(col)
        raise

## Clean BbAHh columns

In [41]:
BbAHh = pd.DataFrame(df[ df['BbAHh'].notnull()]['BbAHh'])

In [51]:
vals = pd.to_numeric(BbAHh['BbAHh'],errors='coerce').notna()
BbAHh[[not item for item in vals]]

,BbAHh
111324,"+0.5,+1"
111325,"-1,-1.5"
111326,"0,+0.5"
111327,"-1,-1.5"
111330,"+0.5,+1"
111331,"-1,-1.5"
111336,"-0.5,-1"
111341,"-0.5,-1"
111342,"0,-0.5"
111343,"-1.5,-2"


In [64]:
def conv_BbAHh(i):
    try:
        return float(i)
    except:
        # print(i)
        a, b = map(float, i.split(','))
        # print(a, b)
        return (a + b) / 2


try:
    df['BbAHh'] = df['BbAHh'].apply(conv_BbAHh)
except:
    pass

print("Final dataframe shape:", df.shape)
print(df.head(10))

Final dataframe shape: (179803, 172)
      Season             Datetime  Div   Country          League   Referee  \
0       2020  2020-08-21 00:30:00  USA       USA             MLS        -1   
1       2020  2020-08-21 00:30:00  USA       USA             MLS        -1   
2       2020  2020-08-21 00:00:00  BRA    Brazil         Serie A        -1   
3       2020  2020-08-21 00:00:00  USA       USA             MLS        -1   
4       2020  2020-08-21 00:00:00  BRA    Brazil         Serie A        -1   
5       2020  2020-08-20 23:15:00  BRA    Brazil         Serie A        -1   
6  2020/2021  2020-08-20 19:30:00  SC0  Scotland  Premier League  W Collum   
7       2020  2020-08-20 13:00:00  CHN     China    Super League        -1   
8       2020  2020-08-20 11:00:00  CHN     China    Super League        -1   
9       2020  2020-08-20 01:30:00  BRA    Brazil         Serie A        -1   

                 HomeTeam            AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
0           Columbus Cr

## Build DF without time(for ML Method)

Due to the features of ML Model, we use the odds to predict the probability of the game result and accroding to the following formula:


We can calculate the final odds.

So the features for prediction is without time sequence and the desired result is the final result of the game

In [75]:
ML_X = df.drop(
    set(results + match_statistics +
        ['Date','Time','Datetime', 'Season', 'HT', 'AT']).intersection(column_names),
    axis=1)
print(ML_X.shape)

# ground truth(match result)
ML_Y = df['FTR']

#
ce_binaryX = ce.BinaryEncoder(
    cols=['HomeTeam', 'AwayTeam', 'Div', 'League', 'Country'])
ohe = ce.OneHotEncoder(cols=[])
ML_X = ce_binaryX.fit_transform(ML_X)
ML_X = ohe.fit_transform(ML_X)

# Encode the string columns
label_encoder = LabelEncoder()
label_encoder.fit(['H', 'D', 'A'])

ML_Y = label_encoder.fit_transform(ML_Y)

print(ML_X.shape)

(179803, 138)
(179803, 177)


## Build DF with time(for LSTM)

X: Features with time sequence

Y: odds of B365


In [77]:
seq_X = df.drop(
    set(results + match_statistics +
        ['Datetime', 'Season', 'HT', 'AT']).intersection(column_names),
    axis=1)
print(seq_X.shape)

# ground truth:odds
seq_Y = df['B365H']

#
ce_binaryX = ce.BinaryEncoder(
    cols=['HomeTeam', 'AwayTeam', 'Div', 'League', 'Country'])

ohe = ce.OneHotEncoder(cols=[])
seq_X = ce_binaryX.fit_transform(seq_X)
seq_X = ohe.fit_transform(seq_X)

print(seq_X.shape)

(179803, 140)
(179803, 179)


# TBD(BELOW THIS LINE)--------------------------------------------------------------------

# Unsupervised Learning

We then proceed to the unsupervised learning section.

In [ ]:
seed = 153

# 80-20 split on training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.cluster import SpectralClustering
import matplotlib.pyplot as plt
from hmmlearn import hmm # Hidden Markovnikov model

scaler = StandardScaler()
pca = PCA()
pca10 = PCA(n_components=10)
pipeline = make_pipeline(scaler,pca)
pipeline10 = make_pipeline(scaler,pca10)

pipeline.fit(X_train)
pipeline10.fit(X_train)

# Plot the explained variances of the first 10 components
features = range(pca10.n_components_)
plt.bar(features, pca10.explained_variance_)
plt.xlabel('PCA feature')
plt.ylabel('variance')
plt.xticks(features)
plt.show()

It turns out that from the graph above, only 4 components of the PCA reduction have significant variance. Hence, we will use 4 dimensions in our reduced dimensionality space. 

We tried to scale the input and fit the data but to no avail. The clustering turns out to be unable to differentiate between wins, draws and losses. With regards to the score, kmeans with PCA and normaliser is the best, even though it is still a poor fit since the score is extremely negative (0 being the best).

In [ ]:
# shown above that only first 4 PCA features have significant variance
pca = PCA(n_components=4)
scaler = StandardScaler()
normalizer = Normalizer()
kmeans = KMeans(n_clusters=3)

kpipeline = make_pipeline(pca,kmeans)
spipeline = make_pipeline(pca,scaler,kmeans)
npipeline = make_pipeline(pca,normalizer,kmeans)

kmeans.fit(X_train)
kpipeline.fit(X_train)
spipeline.fit(X_train)
npipeline.fit(X_train)

In [ ]:
pca = PCA(n_components=4)
scaler = StandardScaler()
normalizer = Normalizer()
#sc = SpectralClustering(3, assign_labels='discretize')

scpipeline = make_pipeline(pca,sc)
sscpipeline = make_pipeline(pca,scaler,sc)
nscpipeline = make_pipeline(pca,normalizer,sc)

scpipeline.fit(X_train)
sscpipeline.fit(X_train)
nscpipeline.fit(X_train)

In [ ]:
pca = PCA(n_components=4)
normalizer = Normalizer()
gm = hmm.GaussianHMM(n_components=3)

ngmpipeline = make_pipeline(pca,normalizer,gm)

ngmpipeline.fit(X_train)

In [ ]:
# The presence of PCA improves raw performance by reducing the number of computations for features with low variance

print("Outcome Legend (Unsupervised Learning)")
print("0: Away win")
print("1: Draw")
print("2: Home win")
print("Note: Supervised learning uses a different outcome legend\n")

kmeans.fit(X_train, y_train)
labels = kmeans.predict(X_test)
df1 = pd.DataFrame({'labels': labels, 'wins': y_test})
ct = pd.crosstab(df1['labels'],df1['wins'])
print('kmeans only')
print(ct)
print("Score:", kmeans.score(X_test, y_test))
print()

kpipeline.fit(X_train, y_train)
labels = kpipeline.predict(X_test)
df1 = pd.DataFrame({'labels': labels, 'wins': y_test})
ct = pd.crosstab(df1['labels'],df1['wins'])
print('kmeans with PCA')
print(ct)
print("Score:", kpipeline.score(X_test, y_test))
print()

spipeline.fit(X_train, y_train)
labels = spipeline.predict(X_test)
df1 = pd.DataFrame({'labels': labels, 'wins': y_test})
ct = pd.crosstab(df1['labels'],df1['wins'])
print('kmeans with PCA & scaler')
print(ct)
print("Score:", spipeline.score(X_test, y_test))
print()

npipeline.fit(X_train, y_train)
labels = npipeline.predict(X_test)
df1 = pd.DataFrame({'labels': labels, 'wins': y_test})
ct = pd.crosstab(df1['labels'],df1['wins'])
print('kmeans with PCA & normalizer')
print(ct)
print("Score:", npipeline.score(X_test, y_test))
print()

In [ ]:
# F1 = 2 * (precision * recall) / (precision + recall)
# Using weighted average to account for label imbalance
from sklearn.metrics import f1_score, accuracy_score

kmeans.fit(X_train, y_train)
labels = kmeans.predict(X_test)
print('kmeans')
print('f1 score: '+ str(f1_score(y_test, labels, average='weighted')))
print('accuracy: '+ str(accuracy_score(y_test, labels)))
print()

kpipeline.fit(X_train, y_train)
labels = kpipeline.predict(X_test)
print('kmeans with PCA')
print('f1 score: '+ str(f1_score(y_test, labels, average='weighted')))
print('accuracy: '+ str(accuracy_score(y_test, labels)))
print()

spipeline.fit(X_train, y_train)
labels = spipeline.predict(X_test)
print('kmeans with PCA & scaler')
print('f1 score: '+ str(f1_score(y_test, labels, average='weighted')))
print('accuracy: '+ str(accuracy_score(y_test, labels)))
print()

npipeline.fit(X_train, y_train)
labels = npipeline.predict(X_test)
print('kmeans with PCA & normalizer')
print('f1 score: '+ str(f1_score(y_test, labels, average='weighted')))
print('accuracy: '+ str(accuracy_score(y_test, labels)))
print()

'''
#ngmpipeline.fit(X_train, y_train)
labels = ngmpipeline.fit_predict(X_test)
print('hmm with PCA & normalizer')
print('f1 score: '+ str(f1_score(y_test, labels, average='weighted')))
print('accuracy: '+ str(accuracy_score(y_test, labels)))
print()
'''

print('Hence shown that hmm with PCA & normalizer performs the worst')

# Supervised Learning

We then proceed to the supervised learning section

In [ ]:
from sklearn.ensemble import RandomForestClassifier

t1 = time.time()

rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

rfc_pred = rfc.predict(X_test)

print("Accuracy:", rfc.score(X_test, y_test))

t2 = time.time()
print("Time taken:", t2-t1, "seconds")

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

t1 = time.time()
etc = ExtraTreesClassifier()
etc.fit(X_train, y_train)

etc_pred = etc.predict(X_test)

print("Accuracy:", etc.score(X_test, y_test))
t2 = time.time()
print("Time taken:", t2-t1, "seconds")

In [ ]:
from sklearn.linear_model import LogisticRegression

print("WARNING! This may take 10 minutes or so on Kaggle (2020)")

t1 = time.time()

for iters in range(10,201,10):
    lr = LogisticRegression(max_iter=iters)
    lr.fit(X_train, y_train)

    lr_pred = lr.predict(X_test)

    print("Number of iterations:",  iters, "of 200")
    #print(lr.score(X_test, y_test))

    t2 = time.time()
    print("Time taken:", t2-t1, "seconds")
print(lr.score(X_test, y_test))

In [ ]:
print("Random Forest Classifier")
print(rfc.predict_proba(X_test)[:10])

In [ ]:
print("Extra Trees Classifier")
print(etc.predict_proba(X_test)[:10])

In [ ]:
print("Logistic Regression")
print(lr.predict_proba(X_test)[:10])

In [ ]:
pred = {'RandomForestClassifier' : rfc_pred, 'ExtraTreesClassifier': etc_pred, 'LogisticRegression': lr_pred,
        'Actual': y_test}
res_df = pd.DataFrame(pred, columns = ['RandomForestClassifier', 'ExtraTreesClassifier', 'LogisticRegression',
                                       'Actual'])
print(res_df.head(10))

In [ ]:
from sklearn.metrics import confusion_matrix
print("Outcome Legend")
print("0: Away win")
print("1: Draw")
print("2: Home win")
print()
print("Confusion matrix for Random Forest Classifier")
print(confusion_matrix(y_test, res_df['RandomForestClassifier']))
print("Confusion matrix for Extra Trees Classifier")
print(confusion_matrix(y_test, res_df['ExtraTreesClassifier']))
print("Confusion matrix for Logistic Regression")
print(confusion_matrix(y_test, res_df['LogisticRegression']))

Let us combine the models and check the performance.

In [ ]:
from sklearn.ensemble import VotingClassifier
t1 = time.time()
eclf1 = VotingClassifier(estimators=[('1', rfc), ('2', etc), ('3', lr)], voting='hard')
eclf1.fit(X_train, y_train)
print("Accuracy:", eclf1.score(X_test, y_test))
t2 = time.time()
print("Time taken:", t2-t1, "seconds")

In [ ]:
from sklearn.ensemble import VotingClassifier
t1 = time.time()
eclf2 = VotingClassifier(estimators=[('1', rfc), ('2', etc), ('3', lr)], voting='soft')
eclf2.fit(X_train, y_train)
print("Accuracy:", eclf2.score(X_test, y_test))
t2 = time.time()
print("Time taken:", t2-t1, "seconds")

In [ ]:
print("Confusion matrix for Voting Classifier (hard voting)")
print(confusion_matrix(y_test, eclf1.predict(X_test)))
print("Confusion matrix for Voting Classifier (soft voting)")
print(confusion_matrix(y_test, eclf2.predict(X_test)))

We also tried the TensorFlow machine learning framework, but it failed terribly.

In [ ]:
import tensorflow as tf

print(X_train.shape)

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(X_train.shape[1],)),
  tf.keras.layers.Dense(128, activation='tanh'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(3)
])

X_train_numpy = np.asarray(X_train)
y_train_numpy = np.asarray(y_train)
X_test_numpy = np.asarray(X_test)
y_test_numpy = np.asarray(y_test)

predictions = model(np.ndarray.astype(X_train_numpy[:1], np.float32)).numpy()

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

pre_training_result = model.evaluate(np.ndarray.astype(X_test_numpy, np.float32), 
                                     np.ndarray.astype(y_test_numpy, np.float32), verbose=2)
print("Pre-training loss:", pre_training_result[0])
print("Pre-training accuracy:", pre_training_result[1])

model.fit(np.ndarray.astype(X_train_numpy, np.float32),
          np.ndarray.astype(y_train_numpy, np.float32), epochs=10, validation_split=0.2, batch_size=32)

probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

predictions = model(np.ndarray.astype(X_test_numpy, np.float32)).numpy()
forecast = np.argmax(predictions, axis=1)

post_training_result = model.evaluate(np.ndarray.astype(X_test_numpy, np.float32), 
                                      np.ndarray.astype(y_test_numpy, np.float32), verbose=2)
print("Post-training loss:", post_training_result[0])
print("Post-training accuracy:", post_training_result[1])
print()

print("Probability prediction")
print(tf.nn.softmax(predictions).numpy()[:10])
print("Result prediction")
print(forecast[:10])


In [ ]:
print("Confusion matrix: ")
print(confusion_matrix(y_test, forecast))